In [ ]:
%matplotlib inline
import yt
import pyxsim
import yt.units as u
import aplpy
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt

In [ ]:
ds = yt.load("GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0150")

In [ ]:
sp = ds.sphere("c", (500.,"kpc"))

In [ ]:
apec_model = pyxsim.XSpecThermalModel("apec", 0.1, 10.0, 10000)

In [ ]:
source_model = pyxsim.ThermalSourceModel(apec_model, Zmet=0.5)

In [ ]:
photons = pyxsim.PhotonList.from_data_source(sp, 0.1, 2000.0*u.cm**2, 100000.0*u.s, source_model)

In [ ]:
tbabs_model = pyxsim.XSpecAbsorbModel("TBabs", 0.04)

In [ ]:
arf = pyxsim.AuxiliaryResponseFile("/Users/jzuhone/Data/xray_data/acisi_aimpt_cy17.arf")
rmf = pyxsim.RedistributionMatrixFile("/Users/jzuhone/Data/xray_data/acisi_aimpt_cy17.rmf")

In [ ]:
events_z = photons.project_photons("z", area_new=arf, exp_time_new=(50.,"ks"), absorb_model=tbabs_model,
                                   sky_center=(45.,30.))

In [ ]:
events_z.write_fits_image("sloshing_z_img.fits", emin=0.5, emax=7.0, clobber=True)

In [ ]:
fig = aplpy.FITSFigure("sloshing_z_img.fits")
fig.show_grayscale(stretch='sqrt')

In [ ]:
events_z.convolve_energies(rmf)

In [ ]:
events_z.write_spectrum("sloshing_z_spec.fits", clobber=True)

In [ ]:
f = pyfits.open("sloshing_z_spec.fits")
plt.figure(figsize=(10,10))
plt.loglog(f["SPECTRUM"].data["CHANNEL"], f["SPECTRUM"].data["COUNTS"])
plt.xlim(30, 1000)
plt.xlabel("Channel (PI)")
plt.ylabel("Counts")
f.close()